# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортируем библиотеки

In [18]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
import warnings
warnings.filterwarnings("ignore")

Откроем файл и изучим данные

In [19]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


 - пропусков нет
 - можно заменить тип данных в calls и messages на int64, они целочисленные.

In [21]:
df['calls'] = df['calls'].astype('int')
df['messages'] = df['messages'].astype('int')

In [22]:
df.dtypes

calls         int64
minutes     float64
messages      int64
mb_used     float64
is_ultra      int64
dtype: object

In [23]:
df['is_ultra'].value_counts() / df['is_ultra'].shape * 100

0    69.352831
1    30.647169
Name: is_ultra, dtype: float64

Данные в выборке не сбалансированны. 69% "Смарт", 31% - "Ультра"

Изучили данные из таблицы - пропусков нет, заменили тип данных в столбцах calls и messages на int

## Разбейте данные на выборки

Так как спрятанной тестовой выборки у нас нету, разделим исходные данные на три выборки: обучающую, валидационную и тестовую.

Сначала методом train_test_split разделим исходные данные на обучающую (60%) и валидационную выборку (40%). После этого разделим валидационную выборку пополам — на валидационную и тестовую выборки. Таким образом каждая из этих 2-х выборок составит 20% из всех данных df.

In [24]:
df_train, df_valid = train_test_split(df, train_size=0.60, test_size=0.40, random_state=123)
df_valid, df_test = train_test_split(df_valid, train_size=0.50, test_size=0.50, random_state=123)

In [25]:
#добавлено ревьюером
df_train.shape, df_valid.shape, df_test.shape

((1928, 5), (643, 5), (643, 5))

В каждой выборке выделим признаки (features) и целевой признак (target). В данном случае целевым признаком является значения столбца is_ultra, а признаками — значение всех остальных столбцов датафрейма df.

In [26]:
# обучающая выборка
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

# валидационная выборка
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

# тестовая выборка
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Разделили выборки на обучающую, валидационную и тестовую. Выделили признаки и целевой признак

## Исследуйте модели

Так как перед нами стоит задача бинарной классификации — проанализировать поведение клиентов и предложить пользователям новый тариф — на этом этапе рассмотрим следующие модели классификации:

 - дерево принятия решений (Decision Tree Classifier)
 - случайный лес (Random Forest Classifier)
 - логистическая регрессия (Logistic Regression).

Исследуем дерево решений

Проверим модель с изменениями гиперпараметров и выберем модель с наилучшим значением

In [27]:
best_model = None
best_result = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_model = model
        best_result = result
        
print("Accuracy лучшей модели:", best_result)

Accuracy лучшей модели: 0.8227060653188181


In [28]:
best_model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')

Наилучший показатель Accuracy показывает модель со значением max_depth=5

Исследуем случайный лес

In [29]:
best_model_rfc = None
best_result_rfc = 0
for depth in range(1,11):
    for est in range(10, 101, 10):
        model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_rfc.fit(features_train, target_train) 
        result = model_rfc.score(features_valid, target_valid)
        if result > best_result_rfc:
            best_model_rfc = model_rfc
            best_result_rfc = result

print("Accuracy наилучшей модели:", best_result_rfc)

Accuracy наилучшей модели: 0.8367029548989113


In [30]:
best_model_rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=70,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

Наилучшее значение Accuracy у мадели с параметрами max_depth=10 и n_estimators=70

Исследуем логистическую регрессию

In [31]:
model_lr = LogisticRegression(random_state=12345)
model_lr.fit(features_train, target_train) 
result_lr = model_lr.score(features_valid, target_valid) 

print("Accuracy модели логистической регрессии:", result_lr)

Accuracy модели логистической регрессии: 0.7200622083981337


Значение параметра Accuracy у логистической регрессии 0.72 не дотягивает для целевого и случайного леса 0.8367. Даже ниже чем у дерева решений 0.82, поэтому эту модель использовать не будем. На тестовой выборке проверим случайный лес.

## Проверьте модель на тестовой выборке

Проверим случайный лес на тестовой выборке

In [32]:
model_final_rfc = RandomForestClassifier(random_state=123, n_estimators=70, max_depth=10)
model_final_rfc.fit(features_train, target_train)  
predictions_final_rfc = model_final_rfc.predict(features_test)

accuracy_final_rfc = accuracy_score(target_test, predictions_final_rfc)
print("Accuracy модели случайного леса:", accuracy_final_rfc)

Accuracy модели случайного леса: 0.80248833592535


Вывод. Победил случайный лес с результатом 80%, что соответствует цели проекта.

## (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность будем использовать классификатор DummyClassifier из библиотеки Skelarn.

Для проверки модели отберем 3 стратегии — stratified, most_frequent и uniform. Если их способность предсказать результат будет лучше чем у настроенной модели случайного леса, будем считать, что наша модель работает плохо и ее необходимо настраивать заного.

In [33]:
strategies = ['stratified', 'most_frequent', 'uniform'] 
  
dummy_results = [] 
for strategy in strategies: 
    dc = DummyClassifier(strategy = strategy, random_state = 42)
    
    dc.fit(features_train, target_train) 
    result = dc.score(features_test, target_test) 
    dummy_results.append({strategy: result}) 

pd.DataFrame(dummy_results)

,stratified,most_frequent,uniform
0,0.578538,NaN,NaN
1,NaN,0.693624,NaN
2,NaN,NaN,0.475894


У финальной модели случайного леса показатель accuracy_score был 80%, что гораздо выше, чем те показатели, которые выдал классификатор DummyClassifier(). Соответсвенно, можем сделать вывод, что модель случайного леса работает достаточно хорошо и ее можно использовать на практике.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
